In [ ]:
using Random, LinearAlgebra, SparseArrays
using Graphs, MetaGraphs, GraphPlot

using MagneticLaplacianSparsifier
using MagneticLaplacianSparsifier: getRNG

In [ ]:
# graph generation
n = 30
p = 0.5
eta = 0.3

rng = getRNG(0)
comp_graph = gen_graph_mun(rng, n, p, eta)  # add rng
collect(edges(comp_graph))

# plotting
gplot(comp_graph, nodelabel=1:nv(comp_graph))

# sample a CRSF
q = 0
crsf = multi_type_spanning_forest(rng, comp_graph, q)

flt_branches = collect(Iterators.flatten(get_prop(crsf, :branches)))
flt_cycles = collect(Iterators.flatten(get_prop(crsf, :cycle_nodes)))

print("peeling leafs order: ", flt_branches, "\n")
print("cycle nodes: ", flt_cycles)

# plotting
gplot(crsf, nodelabel=1:nv(crsf))

In [ ]:
# construct Laplacian
B = sp_magnetic_incidence(crsf; oriented=true)
L = B*B'
# reorder Laplacian
ind = [flt_branches; setdiff(1:n,flt_branches)]
L_perm = L[ind,ind]
chol_matrix = sparse(cholesky(L_perm).L)
# number of diag entries
print("choleski off diagonal entries: ", nnz(chol_matrix) - n, "\n")

# upper bound on number of non-zero entries (Prop 2 in paper)
bound = n + sum(length(c) - 3 for c in flt_cycles)
print("theory: upper bound on off diagonal entries:  ", bound)
chol_matrix

In [ ]:
# cholesky of raw Laplacian
sparse(cholesky(L).L)